In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:0000:01


In [3]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.1.2+cpu
tiktoken version: 0.7.0


In [4]:
import urllib

# URL of the PDF file
url = "https://ag.gov.np/files/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf"

# Open the URL and download the PDF
response = urllib.request.urlopen(url)
pdf_data = response.read()

# Save the PDF to a local file
with open("Constitution-of-Nepal_2072_Eng.pdf", "wb") as f:
    f.write(pdf_data)

print("PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'")


PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'


In [5]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00ta 0:00:01


In [6]:
import PyPDF2

# Open the downloaded PDF file
with open("Constitution-of-Nepal_2072_Eng.pdf", "rb") as file:
    # Initialize a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page of the PDF
    raw_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        raw_text += page.extract_text()

# Print the first 500 characters to preview the content
print(raw_text[:50])


 
1 
  
 
 
 
THE CONSTITUTION OF NEPAL  
   
2 
 


In [7]:
print(len(raw_text))

344782


**Split text into tokens**

In [8]:
import re
preprocessed = re.split(r'([,.:;?_!-"()\']|--|\s|[@#%&])', raw_text)

preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['1', 'THE', 'CONSTITUTION', 'OF', 'NEPAL', '2', 'Table', 'of', 'Content', 's', 'Preamble', 'Part-1', 'Preliminary', 'Part-2', 'Citizenship', 'Part-3', 'Fundamental', 'Rights', 'and', 'Duties', 'Part-4', 'Directive', 'Principles', ',', 'Policies', 'and', 'Obligations', 'of', 'the', 'State']


In [9]:
print(len(preprocessed))

65101


In [10]:
#Unique tokens
len(set(preprocessed))

4525

**Create a vocabulary**

In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

4525


In [12]:
#Give indexes and put that as dictionary
vocab = {token:integer for integer,token in enumerate(all_words)}

# Simple Tokenization of text****

In [13]:
"""Putting all together"""

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [14]:
tokenizer = SimpleTokenizerV1(vocab)
tokenizer.encode(""" """)


[]

# Using Bytepair Encoding 

In [15]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [16]:
tokenizer = tiktoken.get_encoding("gpt2")

In [17]:
text = (
    "No law shall be made providing for the death penalty to any one<|endoftext|> Every citizen"
     "of"
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[2949, 1099, 2236, 307, 925, 4955, 329, 262, 1918, 7389, 284, 597, 530, 50256, 3887, 9511, 1659]


In [18]:
tokenizer.encode("fddfsdfsf")

[69, 1860, 9501, 7568, 28202]

In [19]:
tokenizer.decode([1860])

'dd'

**Data Sampling with a sliding window**

**We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict**

In [20]:
import PyPDF2
# Open the downloaded PDF file
with open("Constitution-of-Nepal_2072_Eng.pdf", "rb") as file:
    # Initialize a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page of the PDF
    raw_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        raw_text += page.extract_text()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

86531


In [30]:

vocab_size = len(set(enc_text))
print(vocab_size)

5305


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [23]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

**Test a dataloader with a batch size of 1 and context size of 4**

In [24]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[220, 198,  16, 220]]), tensor([[198,  16, 220, 198]])]


In [25]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[198,  16, 220, 198]]), tensor([[ 16, 220, 198, 220]])]


In [26]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  220,   198,    16,   220],
        [  198,   220,   220,   198],
        [  220,   198,   220,   198],
        [  220,   198, 10970,  7102],
        [ 2257,  2043, 35354,  3963],
        [  399,  8905,  1847,   220],
        [  220,   198,   220,   220],
        [  220,   198,    17,   220]])

Targets:
 tensor([[  198,    16,   220,   198],
        [  220,   220,   198,   220],
        [  198,   220,   198,   220],
        [  198, 10970,  7102,  2257],
        [ 2043, 35354,  3963,   399],
        [ 8905,  1847,   220,   220],
        [  198,   220,   220,   220],
        [  198,    17,   220,   198]])


**Create token embeddings**

# Coding an LLM Architecture

In [27]:
GPT_CONFIG_124M = {
    "vocab_size": 5305,    # Vocabulary size from BPE
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": True       # Query-Key-Value bias
}